In [143]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import numpy
from random import shuffle
from sklearn.linear_model import LogisticRegression
import csv

In [90]:
st = "  I am n't feeling well :::----;;;;;''''''' and don't k,,,,now how to get out of this      "
import re
s = re.sub(r'[^\w\s]','',st)
s.strip()

'I am nt feeling well  and dont know how to get out of this'

In [60]:
import glob
i = 0
for filename in glob.glob('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/neg/*.txt', recursive=True):
    with open(filename) as myfile:
        data="".join(line.rstrip() for line in myfile)
        data = re.sub(r'[^\w\s]','',data)
        data = data.strip()
    with open(filename, 'w') as out:
        out.write(data + '\n')
    with open('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/neg/train-neg.txt','a') as out:
        out.write(data + '\n')
    if i <= 49:
        with open('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/neg/test-neg.txt','a') as out:
            out.write(data + '\n')
        i += 1
        



In [62]:
i = 0
for filename in glob.glob('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/pos/*.txt', recursive=True):
    with open(filename) as myfile:
        data="".join(line.rstrip() for line in myfile)
        data = re.sub(r'[^\w\s]','',data)
        data = data.strip()
    with open(filename, 'w') as out:
        out.write(data + '\n')
    with open('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/pos/train-pos.txt','a') as out:
        out.write(data + '\n')
    if i <= 49:
        with open('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/pos/test-pos.txt','a') as out:
            out.write(data + '\n')
        i += 1
        

In [63]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        reverse = {}
        for key, value in sources.items():
            if value not in reverse:
                reverse[value] = [key]
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [75]:
sources = {'/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/neg/train-neg.txt':'TRAIN_NEG', '/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/pos/train-pos.txt':'TRAIN_POS','/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/neg/test-neg.txt':'TEST_NEG','/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/pos/test-pos.txt':'TEST_POS'}
sentences = LabeledLineSentence(sources)

In [76]:
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)
model.build_vocab(sentences.to_array())

In [77]:
for epoch in range(10):
    model.train(sentences.sentences_perm())

In [78]:
model.save('./imdb.d2v')

In [79]:
model = Doc2Vec.load('./imdb.d2v')

In [99]:
model.docvecs['TRAIN_NEG_1'].shape


(100,)

In [82]:
train_arrays = numpy.zeros((2000, 100))
train_labels = numpy.zeros(2000)
for i in range(1000):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_arrays[1000 + i] = model.docvecs[prefix_train_neg]
    train_labels[i] = 1
    train_labels[1000 + i] = -1

In [85]:
test_arrays = numpy.zeros((100, 100))
test_labels = numpy.zeros(100)

for i in range(50):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_arrays[50 + i] = model.docvecs[prefix_test_neg]
    test_labels[i] = 1
    test_labels[50 + i] = -1

In [86]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [94]:
classifier.score(test_arrays, test_labels)

0.85999999999999999

In [121]:
test_arrays_proba = classifier.predict_proba(test_arrays)

In [216]:
test_arrays_proba_reduced = map(lambda x:(x[0]* -1 + x[1]*1),test_arrays_proba)

In [217]:
sentiment_score = []
for itr in test_arrays_proba_reduced:
    sentiment_score.append(itr)

In [219]:
review_text_test = []
with open('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/pos/test-pos.txt', 'r') as f:
    reader = csv.reader(f, dialect='excel', delimiter='\n')
    for row in reader:
        review_text_test.append(row)


In [220]:
with open('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/neg/test-neg.txt', 'r') as f:
    reader = csv.reader(f, dialect='excel', delimiter='\n')
    for row in reader:
        review_text_test.append(row)

In [221]:
predicted_sentiment_list = classifier.predict(test_arrays).tolist()

In [222]:
actual_sentiment_list = test_labels.tolist()

In [224]:
final_output = [['Review_Text','Actual_Sentiment','Computed_Sentiment','Sentiment_Score']]
for itr in range(0,100):
    final_output.append([review_text_test[itr],actual_sentiment_list[itr],predicted_sentiment_list[itr],sentiment_score[itr]])
    

In [225]:
with open('/home/vamsi/Downloads/Problem1_Dataset/txt_sentoken/report.csv','w') as csvfile:
    writer = csv.writer(csvfile,delimiter = ',')
    writer.writerows(final_output)
